## Day 31 Lecture 2 Assignment

In this assignment, we will learn about the weighting and scaling with the K nearest neighbor algorithm. We will use the wine quality dataset loaded below and analyze the model generated for this dataset.

In [1]:
%matplotlib inline

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
wine = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-science-lectures/master/wineQualityReds.csv', index_col=0)
wine.head()

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


Recall that we need to check for missing data.

In [3]:
# answer below:
wine.isnull().sum()


fixed.acidity           0
volatile.acidity        0
citric.acid             0
residual.sugar          0
chlorides               0
free.sulfur.dioxide     0
total.sulfur.dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

Convert quality to a binary variable, with the dividing line between 5 and 6.

In [4]:
wine.dtypes

fixed.acidity           float64
volatile.acidity        float64
citric.acid             float64
residual.sugar          float64
chlorides               float64
free.sulfur.dioxide     float64
total.sulfur.dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
quality                   int64
dtype: object

In [5]:
wine['quality'] = wine['quality'].map(lambda x: 'High' if x > 5 else 'Low')

In [6]:
wine

,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
1,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,Low
2,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,Low
3,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,Low
4,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,High
5,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,Low
...,...,...,...,...,...,...,...,...,...,...,...,...
1595,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,Low
1596,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,High
1597,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,High
1598,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,Low


Create a train test split with 20% of the data in the test subsample.

In [7]:
# answer below:
from sklearn.model_selection import train_test_split

x = wine.drop(columns='quality')
y = wine['quality']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


Scale only the independent variables using the minmax scaler.

In [8]:
# answer below:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


Create a KNN model with k=5 and report the accuracy scores. Then make a second model using the scaled data and compare your results.

In [9]:
# answer below:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=57)
knn.fit(x_train, y_train)
print('Accuracy score for non-scaled train:', knn.score(x_train, y_train))
print('Accuracy score for non-scaled test:', knn.score(x_test, y_test))


Accuracy score for non-scaled train: 0.6739640344018765
Accuracy score for non-scaled test: 0.6125


In [10]:
x_train_scaled.shape

(1279, 11)

In [11]:
x_test_scaled.shape

(320, 11)

In [12]:
y_train.shape

(1279,)

In [13]:
y_test.shape

(320,)

In [14]:
knn_scaled = knn.fit(x_train_scaled, y_train)

y_test_pred = knn_scaled.predict(x_test_scaled)

print('Accuracy score for scaled train:', knn.score(x_train_scaled, y_train))
print('Accuracy score for scaled train:', knn.score(x_test_scaled, y_test))

Accuracy score for scaled train: 0.7513682564503519
Accuracy score for scaled train: 0.7125


When generating a KNN model, we can use the weighted model by setting `weights='distance'`. We can also write our own custom weights function.

Write a custom weight function that assigns the weight of 1/sqrt(distance) and use this function in your model. Report the accuracy score.

Hint: Use the `_get_weights` function in scikit learn as a resource. The code is <a href="https://github.com/scikit-learn/scikit-learn/blob/fdbaa58acbead5a254f2e6d597dc1ab3b947f4c6/sklearn/neighbors/base.py#L63" title="_get_weights">here</a>.

In [15]:
# answer below:
def _get_weights(dist):
    """Get the weights from an array of distances and a parameter ``weights``
    Parameters
    ----------
    dist : ndarray
        The input distances
    weights : {'uniform', 'distance' or a callable}
        The kind of weighting used
    Returns
    -------
    weights_arr : array of the same shape as ``dist``
        if ``weights == 'uniform'``, then returns None
    """
    if dist.dtype is np.dtype(object):
        for point_dist_i, point_dist in enumerate(dist):
            # check if point_dist is iterable
            # (ex: RadiusNeighborClassifier.predict may set an element of
            # dist to 1e-6 to represent an 'outlier')
            if hasattr(point_dist, '__contains__') and 0. in point_dist:
                dist[point_dist_i] = point_dist == 0.
            else:
                dist[point_dist_i] = 1. / np.sqrt(point_dist)
    else:
        with np.errstate(divide='ignore'):
            dist = 1. / np.sqrt(dist)
        inf_mask = np.isinf(dist)
        inf_row = np.any(inf_mask, axis=1)
        dist[inf_row] = inf_mask[inf_row]
    return dist


In [17]:
def weights_sqrt(distance):
  distance = []
  distance.append(1./np.sqrt(distance))
  return distance



In [16]:
knn2 = KNeighborsClassifier(weights=_get_weights, n_neighbors=57)
knn2.fit(x_train_scaled, y_train)
print('train score:', knn2.score(x_train_scaled, y_train))
print('test score:', knn2.score(x_test_scaled, y_test))

train score: 1.0
test score: 0.790625


In [18]:
knn3 = KNeighborsClassifier(weights=weights_sqrt)
knn3.fit(x_train_scaled, y_train)
print('train score:', knn3.score(x_train_scaled, y_train))
print('test score:', knn3.score(x_test_scaled, y_test))

ValueError: ignored